# 0. Imports

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
import operator as op

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}.')

print ('Setup completed.')

Device: cuda.
Setup completed.
